In [1]:
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import component, pipeline, Artifact, ClassificationMetrics, Input, Output, Model, Metrics, Dataset
from typing import NamedTuple
from google.cloud import aiplatform

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The module `kfp.v2` is deprecated and will be removed in a futureversion. Please import directly from the `kfp` namespace, instead of `kfp.v2`.
  """Entry point for launching an IPython kernel.


In [2]:
project_id = 'qwiklabs-gcp-03-6e0d35a97dd4'
pipeline_root_path = 'gs://pipeline-tester3'

In [3]:
# INGEST THE DATA

In [4]:
@dsl.component(
    packages_to_install=['tensorflow==2.11.0', 'tensorflow-datasets', 'numpy==1.21.6']
)
def ingest_data() -> str:
    import tensorflow_datasets as tfds
    import numpy as np
    import tensorflow as tf

    print("\n\n" + "Tensorflow version is: " + tf.__version__ + "\n\n")
    print("\n\n" + "Tfds version is: " + tfds.__version__ + "\n\n")

    validation_split = 10
    #bucket = 'gs://tfds-dir3'
    #bucket = 'gs://pipeline-tester3'
    bucket = 'gs://workbench-ron-tensor'

    # test_ds, cifar10_info = tfds.load('cifar10', split='test', with_info=True, as_supervised=True, shuffle_files=True, data_dir="gs://tfds-dir")
    # test_ds = tfds.load('cifar10', split='test', as_supervised=True, shuffle_files=True, data_dir=bucket + "/test", try_gcs=True)
    # validation_ds = tfds.load('cifar10', split=f'train[:{validation_split}%]', as_supervised=True, data_dir=bucket + "/valid", try_gcs=True)
    # training_ds = tfds.load('cifar10', split=f'train[{validation_split}%:]', as_supervised=True, data_dir=bucket + "/train", try_gcs=True)

    test_ds = tfds.load('cifar10', split='test', as_supervised=True, shuffle_files=True)
    validation_ds = tfds.load('cifar10', split=f'train[:{validation_split}%]', as_supervised=True)
    training_ds = tfds.load('cifar10', split=f'train[{validation_split}%:]', as_supervised=True)

    # need the "self" parameter as their is an implicit argument in the custom_shard_func
    # that gives an error saying one arg expected but two were given
    def custom_shard_func(self, element):
        return np.int64(0)

    training_ds.save(
        path=bucket + "/train_ds", shard_func=custom_shard_func)

    validation_ds.save(
        path=bucket + "/valid_ds", shard_func=custom_shard_func)

    test_ds.save(
        path=bucket + "/test_ds", shard_func=custom_shard_func)

    return bucket



In [5]:
# CREATE THE MODEL

In [6]:
@dsl.component(
    packages_to_install=['tensorflow==2.11.0', 'keras'],
    base_image='gcr.io/deeplearning-platform-release/tf-gpu.2-11',
)
def create_model(text: str) -> str:
    import tensorflow as tf
    from keras import applications

    #bucket = 'gs://tfds-dir3'
    # bucket = 'gs://pipeline-tester3'
    bucket = 'gs://workbench-ron-tensor'

    # check for GPU:
    print('\n\n GPU name: ', tf.config.experimental.list_physical_devices('GPU'))
    print('\n\n')

    #Multi GPU strategy
    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():
        new_model = tf.keras.Sequential([
            # applications.ResNet50(weights=None, include_top=False, input_shape=(32, 32, 3)),
            tf.keras.layers.InputLayer((32, 32, 3)),
            tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
            tf.keras.layers.MaxPool2D(),
            tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
            tf.keras.layers.MaxPool2D(),
            tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
            tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
            tf.keras.layers.MaxPool2D(),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])

    new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    print('\n\n' + str(new_model.summary()) + '\n\n')
    new_model.save(bucket + "/untrained-model")
    return "model saved:" + bucket


In [7]:
# TRAIN THE MODEL

In [8]:
@dsl.component(
    packages_to_install=['tensorflow==2.11.0', 'tensorflow-datasets'],
    output_component_file="train_model.yaml",
    base_image='gcr.io/deeplearning-platform-release/tf-gpu.2-11',
)
def train_model(text: str) -> str:
    import tensorflow_datasets as tfds
    import numpy as np
    import tensorflow as tf
    import os
    import datetime

    # check for GPU:
    print('\n\n GPU name: ', tf.config.experimental.list_physical_devices('GPU'))
    print('\n\n')

    # env variable that tells tensorboard where to store logs
    #os.environ['AIP_TENSORBOARD_LOG_DIR']
    #os.environ['gs://tensorboard3']

    #Storage buckets
    # bucket = 'gs://tfds-dir3'
    # bucket = 'gs://pipeline-tester3'
    bucket = 'gs://workbench-ron-tensor'
    
    #Multi GPU strategy
    strategy = tf.distribute.MirroredStrategy()

    # batch size
    batch_size = 32 * strategy.num_replicas_in_sync

    # load data from gcs bucket
    model_name = 'simple model'
    train_data = tf.data.Dataset.load(bucket + "/train_ds")
    train_data = train_data.map(lambda f, l: (tf.cast(f, tf.float64) / 255, l))
    train_data = train_data.shuffle(buffer_size=5000)
    train_data = train_data.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    print("\n\n finish loading training data \n\n")

    valid_data = tf.data.Dataset.load(bucket + "/valid_ds")
    valid_data = valid_data.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    print("\n\n finish loading validation data \n\n")
    test_data = tf.data.Dataset.load(bucket + "/test_ds")
    test_data = test_data.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    print("\n\n finish loading test data \n\n")

    # load model from gcs
    model = tf.keras.models.load_model(bucket + '/untrained-model')

    # Create training callbacks
    earlystop = tf.keras.callbacks.EarlyStopping('val_loss', patience=5, restore_best_weights=True)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=bucket + f'/ckpts/cifar10-{model_name}-' + '{epoch:02d}-{val_accuracy:.4f}')
    log_dir = bucket + "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Train the model
    history = model.fit(train_data, validation_data=valid_data, epochs=30, callbacks=[earlystop, checkpoint])
    #history = model.fit(train_data, validation_data=valid_data, epochs=10)
    print('\n\n history\n' + str(history) + '\n\n')

    # Evaluate the model
    test_loss, test_acc = model.evaluate(test_data)
    print('\n\n'  + f'Test accuracy: {test_acc * 100:.2f}%' + '\n\n')

    #Save the model
    model.save(bucket + "/simple_ model")

    return "model trained"


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: output_component_file parameter is deprecated and will eventually be removed. Please use `Compiler().compile()` to compile a component instead.
  


In [9]:
# CREATE THE PIPELINE

In [10]:
@dsl.pipeline(
    name='simple-pipeline',
    description='testing pipeline',
    pipeline_root=pipeline_root_path
)
def workbench_cnn_pipeline(    
    bucket: str = 'gs://workbench-ron-tensor',
    project: str = 'tensor-1-1',
):
    ingestion_task = ingest_data()
    create_model_task = create_model(text=ingestion_task.output).set_accelerator_type('NVIDIA_TESLA_V100').set_cpu_limit('4').set_memory_limit('16G').set_accelerator_limit(4)
    train_model_task_4_V100_GPUs = train_model(text=create_model_task.output)\
        .set_accelerator_type('NVIDIA_TESLA_V100')\
        .set_cpu_limit('2')\
        .set_memory_limit('16G')\
        .set_accelerator_limit(4)\
        .set_display_name('2 x V100 GPUS ')
    train_model_task_2_V100_GPUs = train_model(text=create_model_task.output)\
        .set_accelerator_type('NVIDIA_TESLA_V100')\
        .set_cpu_limit('1')\
        .set_memory_limit('16G')\
        .set_accelerator_limit(2)\
        .set_display_name('1 x V100 GPUS ')

In [11]:
# COMPILE THE PIPELINE TO CREATE THE PIPELINE JSON

In [12]:
if __name__ == '__main__':
    compiler.Compiler().compile(
        pipeline_func=workbench_cnn_pipeline,
        package_path='workbench_cnn_pipeline.json'
    )

In [13]:
aiplatform.init

<bound method _Config.init of <google.cloud.aiplatform.initializer._Config object at 0x7f31fee4ae50>>

In [14]:

job = aiplatform.PipelineJob(display_name = 'workbench_cnn_pipeline',
                             template_path = 'simple-model-pipeline.json',
                             enable_caching=True,
                             )

job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/10566138111/locations/us-central1/pipelineJobs/simple-pipeline-20230423172814
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/10566138111/locations/us-central1/pipelineJobs/simple-pipeline-20230423172814')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/simple-pipeline-20230423172814?project=10566138111


In [ ]:
# ADD MODEL TO MODEL REGISTRY

In [58]:
#Storage buckets
 # bucket = 'gs://tfds-dir3'
bucket = 'gs://pipeline-tester3'
# bucket = 'gs://workbench-ron-tensor'
model_uri = bucket + "/keras_model/keras_ model"
    
my_model = aiplatform.Model.upload(display_name='keras-model', 
                                   artifact_uri=model_uri, 
                                   serving_container_image_uri= 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest')




Creating Model
Create Model backing LRO: projects/10566138111/locations/us-central1/models/9169691680163495936/operations/8430940606418649088
Model created. Resource name: projects/10566138111/locations/us-central1/models/9169691680163495936@1
To use this Model in another session:
model = aiplatform.Model('projects/10566138111/locations/us-central1/models/9169691680163495936@1')


In [ ]:
# DEPLOY MODEL TO ENDPOINT

In [59]:
# my_model = aiplatform.Model("projects/{PROJECT_NUMBER}/locations/us-central1/models/{MODEL_ID}") 
my_model = aiplatform.Model('projects/10566138111/locations/us-central1/models/9169691680163495936@1')


endpoint = my_model.deploy(
     deployed_model_display_name='keras-endpoint',
     traffic_split={"0": 100},
     machine_type="n1-standard-4",
     accelerator_count=0,
     min_replica_count=1,
     max_replica_count=1,
   )

Creating Endpoint
Create Endpoint backing LRO: projects/10566138111/locations/us-central1/endpoints/4942326757085675520/operations/4980761079387783168
Endpoint created. Resource name: projects/10566138111/locations/us-central1/endpoints/4942326757085675520
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/10566138111/locations/us-central1/endpoints/4942326757085675520')
Deploying model to Endpoint : projects/10566138111/locations/us-central1/endpoints/4942326757085675520
Deploy Endpoint model backing LRO: projects/10566138111/locations/us-central1/endpoints/4942326757085675520/operations/4944732282368819200
Endpoint model deployed. Resource name: projects/10566138111/locations/us-central1/endpoints/4942326757085675520


In [ ]:
# GET PREDICTIONS FROM DEPLOYED MODEL

In [104]:
from google.cloud import aiplatform

import numpy as np
from PIL import Image
import base64
from typing import List, Dict
import argparse
import io
import tensorflow as tf

endpoint = aiplatform.Endpoint(
    endpoint_name='projects/10566138111/locations/us-central1/endpoints/4942326757085675520')

path_image = "/home/jupyter/tensor project/tensorflow/workbench_demo/car.jpeg"
img = tf.keras.utils.load_img(
    path_image, target_size=(32, 32))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

model = tf.keras.models.load_model(bucket + "/simple_ model")
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)


image_data = [np.asarray(Image.open(path_image))]
test_image = [(image_data[0] / 255.0).astype(np.float32).tolist()]


endpoint_prediction = endpoint.predict(instances=test_image)
endpoint_score = tf.nn.softmax(predictions[0])

print("\n" + str(endpoint_prediction) + "\n")
print(
    "The Vertex AI Endpoint predicts this image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(endpoint_score)], 100 * np.max(endpoint_score))
)

#endpoint.predict(instances=img_array).predictions




# image_open = open(path_image, 'rb')
# read_image = image_open.read()
# image_decode = tf.image.decode_jpeg(read_image)
# print("This is the Shape of resized image",image_decode.shape)
# resize_image = tf.image.resize(image_decode, [32, 32])
# print("This is the Shape of resized image",resize_image.shape)


# # im = Image.open(IMAGE_PATH)

# x_test = np.asarray(resize_image).astype(np.float32).tolist()

# byte_array = base64.b64decode(path_image)
# image = Image.open(io.BytesIO(byte_array))
# image = np.asarray(image + "=" * 4).astype(np.float32).tolist()

# # image = image / 255.0
# # image = image.reshape(28, 28, 1)


# endpoint.predict(instances=resize_image).predictions


1/1 [==============================] - 0s 85ms/step
This image most likely belongs to automobile with a 23.20 percent confidence.

Prediction(predictions=[[1.24650228, 3.32507133, -3.51667333, -2.77927899, -3.1631844, -4.86116123, -3.0711298, -0.0200937428, -1.09384143, 4.49908733]], deployed_model_id='4629350772239237120', model_version_id='1', model_resource_name='projects/10566138111/locations/us-central1/models/9169691680163495936', explanations=None)

The Vertex AI Endpoint predicts this image most likely belongs to automobile with a 23.20 percent confidence.
